In [1]:
# ==========================================================
# High-Fidelity Artwork Analyzer v13.2 (Stable Local Edition)
# ==========================================================
# Local-only (OpenCV, NumPy, Pillow)
#
# Outputs per image:
#   /<image>_analysis/
#     ├─ overlay_final_composite.png
#     ├─ curatorial_summary.txt
#     └─ metrics.json
# ==========================================================

import os, json, time, textwrap
import numpy as np
import cv2
from PIL import Image
from math import atan2, degrees

# -----------------------------
# Utilities
# -----------------------------
def load_rgb(path):
    img = Image.open(path).convert("RGB")
    return np.array(img)

def ensure_dir(p): os.makedirs(p, exist_ok=True)

def to_uint8(img_f32): return np.clip(img_f32*255.0,0,255).astype(np.uint8)

def safe_gamma(img_f32):
    m = float(img_f32.mean())
    if m < 0.20: img_f32 = np.power(img_f32, 0.6)
    elif m > 0.85: img_f32 = np.power(img_f32, 1.2)
    return np.clip(img_f32, 0, 1)

# -----------------------------
# Color Analysis
# -----------------------------
def ciede2000(Lab1, Lab2):
    L1,a1,b1 = Lab1; L2,a2,b2 = Lab2
    avg_L = (L1 + L2) / 2.0
    C1,C2 = np.sqrt(a1*a1 + b1*b1), np.sqrt(a2*a2 + b2*b2)
    avg_C = (C1 + C2) / 2.0
    G = 0.5*(1 - np.sqrt((avg_C**7)/((avg_C**7)+25**7)))
    a1p,a2p = (1+G)*a1,(1+G)*a2
    C1p,C2p = np.sqrt(a1p*a1p + b1*b1), np.sqrt(a2p*a2p + b2*b2)
    avg_Cp = (C1p + C2p)/2
    h1p = np.degrees(np.arctan2(b1,a1p))%360; h2p = np.degrees(np.arctan2(b2,a2p))%360
    dLp = L2-L1; dCp = C2p-C1p; dhp = h2p-h1p
    if C1p*C2p==0: dhp=0
    else:
        if dhp>180: dhp-=360
        elif dhp<-180: dhp+=360
    dHp = 2*np.sqrt(C1p*C2p)*np.sin(np.radians(dhp)/2)
    avg_Lp=(L1+L2)/2
    if C1p*C2p==0: avg_hp=h1p+h2p
    else:
        if abs(h1p-h2p)>180: avg_hp=(h1p+h2p+360)/2 if (h1p+h2p)<360 else (h1p+h2p-360)/2
        else: avg_hp=(h1p+h2p)/2
    T=(1-0.17*np.cos(np.radians(avg_hp-30))+0.24*np.cos(np.radians(2*avg_hp))
        +0.32*np.cos(np.radians(3*avg_hp+6))-0.20*np.cos(np.radians(4*avg_hp-63)))
    d_ro=30*np.exp(-((avg_hp-275)/25)**2)
    Rc=2*np.sqrt((avg_Cp**7)/((avg_Cp**7)+25**7))
    Sl=1+0.015*(avg_Lp-50)**2/np.sqrt(20+(avg_Lp-50)**2)
    Sc=1+0.045*avg_Cp; Sh=1+0.015*avg_Cp*T; Rt=-np.sin(np.radians(2*d_ro))*Rc
    return float(np.sqrt((dLp/Sl)**2 + (dCp/Sc)**2 + (dHp/Sh)**2 + Rt*(dCp/Sc)*(dHp/Sh)))

def analyze_color_deep(img, k=6):
    H,W = img.shape[:2]
    lab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
    Z = lab.reshape(-1,3).astype(np.float32)
    crit=(cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER,60,0.3)
    _r,labels,centers=cv2.kmeans(Z,k,None,crit,10,cv2.KMEANS_PP_CENTERS)
    labels2 = labels.reshape(H,W)
    centers = centers.astype(np.float32)
    rgb_centers = cv2.cvtColor(centers.reshape(-1,1,3).astype(np.uint8),cv2.COLOR_Lab2RGB).reshape(-1,3)
    palette=[tuple(map(int,c)) for c in rgb_centers]
    counts=np.bincount(labels.flatten(),minlength=k).astype(np.float32)
    masks=[(labels2==i).astype(np.uint8)*255 for i in range(k)]
    cluster_area_frac=counts/(H*W+1e-6)
    spatial_cohesion=float(cluster_area_frac.max())
    p=cluster_area_frac[cluster_area_frac>0]
    palette_entropy=-float(np.sum(p*np.log2(p)))
    hsv=cv2.cvtColor(img,cv2.COLOR_RGB2HSV)
    Hh,Sh,Vh=hsv[:,:,0].astype(np.float32),hsv[:,:,1].astype(np.float32),hsv[:,:,2].astype(np.float32)
    hue_deg=(Hh*2.0)%360.0
    hist,_=np.histogram(hue_deg,bins=36,range=(0,360)); hist=hist.astype(np.float32); hist/=hist.sum()+1e-6
    a,b=centers[:,1]-128,centers[:,2]-128
    hues_c=(np.degrees(np.arctan2(b,a))+360)%360
    warm_ratio=float(((hues_c<90)|(hues_c>330)).sum()/len(hues_c))
    cool_ratio=float(((hues_c>180)&(hues_c<300)).sum()/len(hues_c))
    harmony="mixed"
    if np.ptp(hues_c)<35: harmony="analogous"
    elif any(abs(hues_c[i]-hues_c[j])%360>150 for i in range(len(hues_c)) for j in range(i+1,len(hues_c))):
        harmony="complementary"
    dEs=[ciede2000(centers[i],centers[j]) for i in range(len(centers)) for j in range(i+1,len(centers))]
    mean_dE=float(np.mean(dEs)) if dEs else 0.0; max_dE=float(np.max(dEs)) if dEs else 0.0
    labf=lab.astype(np.float32)
    gxL,gyL=cv2.Sobel(labf[:,:,0],cv2.CV_32F,1,0),cv2.Sobel(labf[:,:,0],cv2.CV_32F,0,1)
    gxa,gya=cv2.Sobel(labf[:,:,1],cv2.CV_32F,1,0),cv2.Sobel(labf[:,:,1],cv2.CV_32F,0,1)
    gxb,gyb=cv2.Sobel(labf[:,:,2],cv2.CV_32F,1,0),cv2.Sobel(labf[:,:,2],cv2.CV_32F,0,1)
    grad_mag=np.sqrt((gxL**2+gyL**2)+(gxa**2+gya**2)+(gxb**2+gyb**2))
    contrast_map=(grad_mag-grad_mag.min())/(grad_mag.max()-grad_mag.min()+1e-6)
    return dict(palette_rgb=palette,masks=masks,color_variability=float(np.std(centers,0).mean()),
                warm_ratio=warm_ratio,cool_ratio=cool_ratio,harmony=harmony,
                hue_hist=hist,spatial_cohesion=spatial_cohesion,
                palette_entropy=palette_entropy,mean_deltaE=mean_dE,max_deltaE=max_dE,
                contrast_map=contrast_map)

# -----------------------------
# Texture, Saliency, Composition, Illumination
# -----------------------------
def gabor_energy(gray):
    H,W=gray.shape; angles=np.linspace(0,180,6,endpoint=False)
    ori_energy=[]
    combined=np.zeros((H,W),np.float32)
    for theta in angles:
        theta_rad=np.deg2rad(theta)
        e_sum=np.zeros((H,W),np.float32)
        for k in (3,5,7):
            kernel=cv2.getGaborKernel((21,21),sigma=k,theta=theta_rad,lambd=10,gamma=0.5,psi=0)
            resp=cv2.filter2D(gray,cv2.CV_32F,kernel); e_sum+=resp*resp
        ori_energy.append(e_sum.mean()); combined=np.maximum(combined,e_sum)
    ori_energy=np.array(ori_energy); dom_angle=float(angles[int(np.argmax(ori_energy))])
    anisotropy=float((ori_energy.max()-ori_energy.min())/(ori_energy.mean()+1e-6))
    return angles,ori_energy,(combined-combined.min())/(combined.max()-combined.min()+1e-6),anisotropy,dom_angle

def analyze_texture(img):
    g=cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    hist,_=np.histogram(g,bins=256,range=(0,256)); hist=hist.astype(np.float32); hist/=hist.sum()+1e-6
    lbp_entropy=-float(np.sum(hist*np.log2(hist+1e-12))); lbp_roughness=float(g.std())
    angles,e_map,gabor_map,anisotropy,dom=gabor_energy(g)
    return dict(lbp_entropy=lbp_entropy,lbp_roughness=lbp_roughness,gabor_angles=angles,
                gabor_energy=e_map,gabor_map=gabor_map,gabor_anisotropy=anisotropy,
                gabor_dom_angle=dom)

def spectral_saliency(grayf):
    fft=np.fft.fft2(grayf); loga=np.log(np.abs(fft)+1e-8)
    spec=np.exp((loga-cv2.blur(loga,(3,3)))+1j*np.angle(fft))
    sal=np.abs(np.fft.ifft2(spec))**2; sal=cv2.GaussianBlur(sal,(9,9),2)
    sal=(sal-sal.min())/(sal.max()-sal.min()+1e-6)
    return (sal*255).astype(np.uint8)

def subject_from_saliency(sal):
    H,W=sal.shape
    _,th=cv2.threshold(sal,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    cnts,_=cv2.findContours(th,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    if cnts: x,y,w,h=cv2.boundingRect(max(cnts,key=cv2.contourArea))
    else: x,y,w,h=W//4,H//4,W//2,H//2
    mask=np.zeros((H,W),np.uint8); cv2.rectangle(mask,(x,y),(x+w,y+h),255,-1)
    return dict(mask=mask,bbox=(x,y,w,h))

def analyze_saliency(img):
    g=cv2.cvtColor(img,cv2.COLOR_RGB2GRAY).astype(np.float32)
    sal=spectral_saliency(g); sub=subject_from_saliency(sal)
    Y,X=np.indices(sal.shape); w=sal.astype(np.float32)+1
    cx,cy=int((X*w).sum()/w.sum()),int((Y*w).sum()/w.sum())
    return dict(saliency_map=sal,focal=(cx,cy),subject_mask=sub["mask"],subject_bbox=sub["bbox"])

def analyze_composition(img,focal_xy):
    g=cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    H,W=g.shape; edges=cv2.Canny(g,100,200)
    linesP=cv2.HoughLinesP(edges,1,np.pi/180,100,minLineLength=min(H,W)//10,maxLineGap=15)
    lines=[tuple(map(int,l[0])) for l in linesP] if linesP is not None else []
    fx,fy=focal_xy; golden=0.618
    thirds_pts=[(W/3,H/3),(2*W/3,H/3),(W/3,2*H/3),(2*W/3,2*H/3)]
    golden_pts=[(golden*W,golden*H),((1-golden)*W,golden*H),(golden*W,(1-golden)*H),((1-golden)*W,(1-golden)*H)]
    diag=np.hypot(W,H)
    thirds_delta=min(np.hypot(fx-x,fy-y) for (x,y) in thirds_pts)/(diag+1e-6)
    golden_delta=min(np.hypot(fx-x,fy-y) for (x,y) in golden_pts)/(diag+1e-6)
    g_f=g.astype(np.float32)/255.0
    left=g_f[:,:W//2]; right=np.fliplr(g_f[:,W-W//2:])
    mse=float(np.mean((left-right)**2)); symmetry=float(1.0-min(1.0,mse*4.0))
    lw,rw=float(np.sum(edges[:,:W//2])),float(np.sum(edges[:,W//2:]))
    balance=(rw-lw)/(lw+rw+1e-6)
    return dict(lines=lines,edge_balance=balance,thirds_delta=thirds_delta,golden_delta=golden_delta,
                symmetry=symmetry,vanishing_point=(W//2,H//2))

def illumination(img):
    L=cv2.cvtColor(img,cv2.COLOR_RGB2LAB)[:,:,0].astype(np.float32)
    mu,sig=L.mean(),L.std()+1e-6; skew=float((((L-mu)/sig)**3).mean())
    gx,gy=cv2.Sobel(L,cv2.CV_32F,1,0),cv2.Sobel(L,cv2.CV_32F,0,1)
    hi=(L>np.percentile(L,90)).astype(np.uint8)
    if hi.sum()>0:
        gx,gy=(gx*hi).sum(),(gy*hi).sum()
        ang=float((degrees(atan2(-gy,-gx))+360)%360)
    else: ang=None
    return dict(lightness_skew=skew,light_direction_deg=ang)
# -----------------------------
# Curatorial Summary Generator
# -----------------------------
def generate_curatorial_summary(M):
    C, T, P, I = M["color"], M["texture"], M["composition"], M["illumination"]

    tone = {
        "analogous": "a restrained, harmonious palette",
        "complementary": "a lively chromatic counterpoint",
    }.get(C["harmony"], "a balanced and nuanced color range")

    if C["mean_deltaE"] > 25:
        contrast = "strong contrast that energizes the composition"
    elif C["mean_deltaE"] > 10:
        contrast = "moderate contrast that keeps the eye moving"
    else:
        contrast = "gentle tonal transitions that calm the surface"

    texture_phrase = (
        f"Texture reveals directional force (anisotropy {T['gabor_anisotropy']:.2f}) "
        f"with a dominant orientation near {T['gabor_dom_angle']:.0f}°."
        if T["gabor_anisotropy"] > 0.5
        else "Texture remains even and diffused, emphasizing balance over motion."
    )

    comp_phrase = (
        f"The focal point aligns with golden proportion (Δ={P['golden_delta']:.3f})."
        if P["golden_delta"] < 0.12
        else f"The composition loosely follows rule-of-thirds (Δ={P['thirds_delta']:.3f})."
        if P["thirds_delta"] < 0.12
        else "The composition resists classical proportion, encouraging open visual flow."
    )

    light_phrase = (
        f"Light enters from around {int(I['light_direction_deg'])}°, shaping perception."
        if I["light_direction_deg"] is not None
        else "Light diffuses evenly, creating tonal unity."
    )

    if I["lightness_skew"] > 0.5:
        mood = "luminous and expansive"
    elif I["lightness_skew"] < -0.5:
        mood = "dark-hued and contemplative"
    else:
        mood = "balanced and calm"

    micro = textwrap.fill(
        " ".join([
            f"Color: {C['harmony']} harmony, mean ΔE={C['mean_deltaE']:.1f}, max ΔE={C['max_deltaE']:.1f}, "
            f"entropy={C['palette_entropy']:.2f}, spatial cohesion={C['spatial_cohesion']:.2f}.",
            f"Texture: entropy={T['lbp_entropy']:.2f}, anisotropy={T['gabor_anisotropy']:.2f}, "
            f"dom. angle={T['gabor_dom_angle']:.0f}°.",
            f"Composition: thirdsΔ={P['thirds_delta']:.3f}, goldenΔ={P['golden_delta']:.3f}, "
            f"symmetry={P['symmetry']:.2f}, balance={P['edge_balance']:.2f}.",
            f"Light: skew={I['lightness_skew']:.2f}, direction={I['light_direction_deg']}."
        ]),
        width=90
    )

    curatorial = textwrap.fill(
        " ".join([
            f"The work presents {tone} with {contrast}.",
            texture_phrase, comp_phrase, light_phrase,
            f"The overall mood feels {mood}.",
            "Together, these qualities invite a slow, attentive gaze where gesture, color, and balance cohere."
        ]),
        width=90
    )

    return micro, curatorial

# -----------------------------
# Visualization / Composite
# -----------------------------
def make_hue_bar(hist,width=420,height=50):
    bar=np.zeros((height,width,3),np.uint8)
    for x in range(width):
        h=int((x/width)*179)
        rgb=cv2.cvtColor(np.uint8([[[h,200,220]]]),cv2.COLOR_HSV2RGB)[0,0]
        bar[:,x]=rgb
    bins=len(hist); bin_w=max(1,width//bins)
    norm=hist/(hist.max()+1e-6)
    for i,v in enumerate(norm):
        x1=i*bin_w; x2=min(width-1,(i+1)*bin_w-1); h_px=int(v*(height-10))
        cv2.rectangle(bar,(x1,height-1),(x2,height-1-h_px),(0,0,0),-1)
    return bar

def make_orientation_bar(angles,energies,width=420,height=50):
    bar=np.full((height,width,3),240,np.uint8)
    if len(energies)==0: return bar
    e=np.array(energies,dtype=np.float32); e=(e-e.min())/(e.max()-e.min()+1e-6)
    for a,v in zip(angles,e):
        x=int((a/180.0)*width); h_px=int(v*(height-10))
        cv2.rectangle(bar,(x-2,height-1),(x+2,height-1-h_px),(60,60,60),-1)
    cv2.putText(bar,"0°",(5,18),0,0.5,(0,0,0),1)
    cv2.putText(bar,"90°",(width//2-15,18),0,0.5,(0,0,0),1)
    cv2.putText(bar,"180°",(width-50,18),0,0.5,(0,0,0),1)
    return bar

# -----------------------------
# Main Pipeline
# -----------------------------
def analyze_artwork_high_fidelity(path):
    start=time.time()
    img=load_rgb(path)
    color=analyze_color_deep(img)
    texture=analyze_texture(img)
    sal=analyze_saliency(img)
    comp=analyze_composition(img,sal["focal"])
    illum=illumination(img)
    M=dict(color=color,texture=texture,saliency=sal,composition=comp,illumination=illum)

    # --- Build composite ---
    base=img.astype(np.float32)/255.0; H,W=base.shape[:2]
    comp_rgb=base.copy()
    legend_h=160; legend=np.zeros((legend_h,W,3),np.uint8)
    hue_bar=make_hue_bar(color["hue_hist"],width=min(500,max(360,W//2)),height=50)
    ori_bar=make_orientation_bar(texture["gabor_angles"],texture["gabor_energy"],
                                 width=min(500,max(360,W//2)),height=50)
    hb_h,hb_w=hue_bar.shape[:2]; ob_h,ob_w=ori_bar.shape[:2]
    x0=max(0,W-max(hb_w,ob_w)-20)
    hue_bar_resized=hue_bar[:,:min(hb_w,W-x0)]
    ori_bar_resized=ori_bar[:,:min(ob_w,W-x0)]
    # safe placement
    y1,y2=10,10+hb_h; x1,x2=x0,x0+hue_bar_resized.shape[1]
    if y2<=legend.shape[0] and x2<=legend.shape[1]:
        legend[y1:y2, x1:x2] = hue_bar_resized
    y1b, y2b = 10 + hb_h + 5, 10 + hb_h + 5 + ob_h
    x1b, x2b = x0, x0 + ori_bar_resized.shape[1]
    if y2b <= legend.shape[0] and x2b <= legend.shape[1]:
        legend[y1b:y2b, x1b:x2b] = ori_bar_resized

    final = np.vstack([to_uint8(safe_gamma(comp_rgb)), legend])
    final = final.astype(np.uint8)

    # --- Save Outputs ---
    out_dir = os.path.splitext(path)[0] + "_analysis"
    ensure_dir(out_dir)
    out_png = os.path.join(out_dir, "overlay_final_composite.png")
    cv2.imwrite(out_png, final)

    # Curatorial Text
    micro, narrative = generate_curatorial_summary(M)
    out_txt = os.path.join(out_dir, "curatorial_summary.txt")
    with open(out_txt, "w", encoding="utf-8") as f:
        f.write("--- MICRO SUMMARY ---\n")
        f.write(micro + "\n\n--- CURATORIAL SUMMARY ---\n")
        f.write(narrative + "\n")

    # Metrics JSON (safe serialization for NumPy types)
    def make_json_safe(obj):
        if isinstance(obj, dict):
            return {k: make_json_safe(v) for k, v in obj.items()}
        elif isinstance(obj, (list, tuple)):
            return [make_json_safe(x) for x in obj]
        elif isinstance(obj, (np.generic, np.number)):
            return obj.item()
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return obj

    metrics_py = make_json_safe(M)
    out_json = os.path.join(out_dir, "metrics.json")
    with open(out_json, "w", encoding="utf-8") as f:
        json.dump(metrics_py, f, indent=2)

    print(f"✅ Analysis complete for: {os.path.basename(path)}")
    print(f"📁 Results saved to: {out_dir}")
    print(f"⏱️  Elapsed: {time.time() - start:.2f}s")

# -----------------------------
# Run
# -----------------------------
if __name__ == "__main__":
    IMAGE_PATH = "/Users/alievanayasso/Documents/SlowMA/VG_-_skeleton.jpg"
    analyze_artwork_high_fidelity(IMAGE_PATH)

✅ Analysis complete for: VG_-_skeleton.jpg
📁 Results saved to: /Users/alievanayasso/Documents/SlowMA/VG_-_skeleton_analysis
⏱️  Elapsed: 31.96s
